**RotoTrackID for Google Colab**

1. Setup

In [ ]:
!git clone https://github.com/nameshigawa/RotoTrackID
%cd RotoTrackID
!pip install -r requirements.txt

2. Download Model
（Ultralytics YOLO + Meta SAM）

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v8.2.0/yolo11n.pt
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b.pth

3. Upload Video File

In [ ]:
from google.colab import files
uploaded = files.upload()
video = list(uploaded.keys())[0]

4. Tracking and Preview File

In [ ]:
from analyze import run_tracking_for_colab

preview, ids = run_tracking_for_colab(
    video_path,
    out_dir="result",
    model_path="yolo11n.pt"
)

preview, ids

5. Preview ID Video

In [ ]:
from IPython.display import HTML

HTML(f"""
<video width=720 controls>
  <source src="{preview}" type="video/mp4">
</video>
<p>Select ID</p>
""")

6. Selecte ID Number

In [ ]:
from alpha_export import export_id

target_id = 3   # Write ID
out_dir = export_id("result", target_id)
out_dir

7. Download Zip File

In [ ]:
!zip -r roto.zip {out_dir}
from google.colab import files
files.download("roto.zip")